# Generate Genre Vocabularies as SKOS (A Second Test: using ConceptSchemes and Collections)
This notebook generates two SKOS Vocabularies (i.e. `skos:ConceptScheme`) for two historic genre classification schemes.
In this notebook `skos:Collection` are used as well.

Why skos:Collections are needed:

* can we infer that there is such a concept as "Lyrische Dichtungsart" if there is an item in the TOC "Lyrische Dichtungsarten"?
* these might be considered as collections instead of concepts; and then we add concepts; but this will/might break the hierarchy
* concepts and collections are disjoint classes!
* how we go about headings: "Die romantische Canzone, das lyrische Sonett usw." – there are obviously more concepts here: "romantische Canzone", "Lyrisches Sonett"; should this be modeled as a collection or separate concepts on the same level in the upper collection

In [1]:
# import helper classes
from dlod.cidoc import E55Type
from dlod.skos import SkosConceptScheme, SkosConcept, SkosCollection
from rdflib import Graph

## Function to generate skos:Concepts from python data structure

In [2]:
def create_concept_from_dict(data:dict, base_uri:str, concept_scheme:SkosConceptScheme):
    """Takes a dictionary with id, label and narrower concept and create the skos:Concept"""

    concept_uri = base_uri + data["id"]
    concept = SkosConcept(uri=concept_uri)
    concept.skos_pref_label(data["label"], "de")
    concept.skos_in_scheme(concept_scheme)

    if "narrower" in data:
        for item in data["narrower"]:
            narrow_concept = create_concept_from_dict(item, base_uri=base_uri, concept_scheme=concept_scheme)
            concept.skos_narrower(narrow_concept)

    return concept

    

## Johann Joachim Eschenburg: Entwurf einer Theorie und Literatur der schönen Redekünste. Zur Grundlage bei Vorlesungen. 4. Auflage. Berlin/Stettin 1817.

* Epische Dichtungsarten	
    * I. Poetische Erzählung
        * 1. Aesopische Fabel
        * 2. Poetische Erzählung
        * 3. Allegorische Erzählung
    * II. Das Hirtengedicht
    * III. Das Epigramm
    * IV. Die Satire
    * V. Das Lehrgedicht, die beschreibende Poesie, die Epistel
    * VI. Die Elegie
    * VII. Die lyrische Poesie
    * VIII. Das Heldengedicht
    * IX. Der Roman
* Dramatische Dichtungsarten
    * I. Das poetische Gespräch
    * II. Die Heroide
    * III. Die Kantate
    * IV. Das Drama überhaupt
    * V. Das Lustspiel
    * VI. Das Trauerspiel
    * VII. Die Oper 

In [3]:
# create a graph to collect generated pieces of the SKOS
eschenburg_g = Graph()

We need this in a structure that we can work with. Therefore we "convert" it to a list with nested dictionaries. Later on we would need a parser for a YAML file or whatever format it is convenient to generate as input.

### Eschenburg ConceptScheme

In [4]:
eschenburg_base_uri = "https://genre.clscor.io/eschenburg/"

In [5]:
eschenburg = SkosConceptScheme(uri=eschenburg_base_uri)
eschenburg.rdfs_label("Eschenburg: Entwurf einer Theorie und Literatur der schönen Redekünste [Gattungssystem]", "de")

True

In [6]:
#add the triples to the joint graph
# maybe it was not the very best idea to name the concept scheme eschenburg, maybe will refactor later
eschenburg_g = eschenburg_g + eschenburg.dump()

### Eschenburg Concepts
The following data structure is a datafied version of the eschenburg toc included above. It is used to generate the `skos:Concept`s.

In [7]:
eschenburg_raw_data = [

    {
        "id" : "epische_dichtungsart",
        "label" : "Epische Dichtungsart",
        "narrower" : [
                {
                    "id" : "poetische_erzaehlung",
                    "label" : "Poetische Erzählung",
                    "narrower" : [

                            
                            {
                                "id" : "aesopische_fabel",
                                "label" : "Aesopische Fabel"
                            },
                            {
                                "id" : "poetische_erzaehlung",
                                "label" : "Poetische Erzählung"
                            },
                            {
                                "id" : "allegorische_erzaehlung",
                                "label" : "Allegorische Erzählung"
                            }
                                
                             
                        ]
                },
                {
                    "id" : "hirtengedicht",
                    "label" : "Hirtengedicht"
                },
                {
                    "id" : "epigramm",
                    "label" : "Epigramm"
                },
                {
                    "id" : "satire",
                    "label" : "Satire"
                },
                {
                    "id" : "lehrgedicht",
                    "label" : "Lehrgedicht"
                },
                {
                    "id" : "beschreibende_poesie",
                    "label" : "Beschreibende Poesie"
                },
                {
                    "id": "epistel",
                    "label" : "Epistel"
                },
                {
                    "id" : "elegie",
                    "label" : "Elegie"
                },
                {
                    "id" : "lyrische_poesie",
                    "label" : "Lyrische Poesie"
                },
                {
                    "id" : "heldengedicht",
                    "label" : "Heldengedicht"
                },
                {
                    "id" : "roman",
                    "label" : "Roman"
                }
    
            ]
    },

    {
        "id" : "dramatische_dichtungsart",
        "label" : "Dramatische Dichtungsart",
        "narrower" : [
                {
                    "id" : "poetisches_gespraech",
                    "label" : "Poetisches Gespräch"
                },
                {
                    "id" : "heroide",
                    "label" : "Heroide"
                },
                {
                    "id" : "kantate",
                    "label" : "Kantate"
                },
                {
                    "id" : "drama",
                    "label" : "Drama"
                },
                {
                    "id" : "lustspiel",
                    "label" : "Lustspiel"
                },
                {
                    "id" : "trauerspiel",
                    "label" : "Trauerspiel"
                },
                {
                    "id" : "oper",
                    "label" : "Oper"
                }
            ]
    }
    
]

In [8]:
# create skos:Concept s for eschenburg

for item in eschenburg_raw_data:
    top_concept = create_concept_from_dict(item, base_uri=eschenburg_base_uri,concept_scheme=eschenburg)
    eschenburg.skos_has_top_concept(top_concept)

In [9]:
#print(eschenburg.serialize())

In [10]:
# store it
# eschenburg.store(folder = "out", filename = "eschenburg_skos_concepts")
# instead of storing, add it to the joint graph
eschenburg_g = eschenburg_g + eschenburg.dump()

### Some manual interventions to Eschenburg Concepts
What does mean

> IV. Das Drama überhaupt

Don't know, but will add this as an alternative label; 
the rule of thumb could be: if a TOC heading is more than a Noun Phrase we record it as an alt Label
Won't record "Das Hirtengedicht", only "Hirtengedicht" as prefLabel, but "Das Drama überhaupt" is more than just the article + noun.
In the case of Eschenburg this is not such a big deal

In [11]:
drama_ueberhaupt = SkosConcept(uri=eschenburg_base_uri+"drama") # be careful, this MUST result in the URI used above
drama_ueberhaupt.skos_alt_label("Das Drama überhaupt","de")
print(drama_ueberhaupt.serialize())

@prefix skos: <http://www.w3.org/2004/02/skos/core#> .

<https://genre.clscor.io/eschenburg/drama> a skos:Concept ;
    skos:altLabel "Das Drama überhaupt"@de .




In [12]:
# merge into eschenburg graph
eschenburg_g = eschenburg_g + drama_ueberhaupt.dump()

### Eschenburg Collections
A parallel structure that tries to preserve parts of the semantic of the original TOC; we don't use `skos:OrderedCollection` at the moment because `rdf:List` is tricky.

If there is a heading in the TOC that can not strictly mapped to a concept, a collection is created with the heading as prefLabel.

An example follows below:

In [13]:
# V. Das Lehrgedicht, die beschreibende Poesie, die Epistel
eschenburg_lehrgedicht_beschreibende_poesie_epistel = SkosCollection(uri=eschenburg_base_uri + "collection/lehrgedicht_beschr_poesie_epistel")
eschenburg_lehrgedicht_beschreibende_poesie_epistel.skos_pref_label("Das Lehrgedicht, die beschreibende Poesie, die Epistel","de")
eschenburg_lehrgedicht_member_ids = [
    eschenburg_base_uri + "lehrgedicht",
    eschenburg_base_uri + "beschreibende_poesie",
    eschenburg_base_uri + "epistel"]

eschenburg_lehrgedicht_beschreibende_poesie_epistel.skos_member(uris=eschenburg_lehrgedicht_member_ids)

True

In [14]:
print(eschenburg_lehrgedicht_beschreibende_poesie_epistel.serialize())

@prefix skos: <http://www.w3.org/2004/02/skos/core#> .

<https://genre.clscor.io/eschenburg/collection/lehrgedicht_beschr_poesie_epistel> a skos:Collection ;
    skos:member <https://genre.clscor.io/eschenburg/beschreibende_poesie>,
        <https://genre.clscor.io/eschenburg/epistel>,
        <https://genre.clscor.io/eschenburg/lehrgedicht> ;
    skos:prefLabel "Das Lehrgedicht, die beschreibende Poesie, die Epistel"@de .




In [15]:
#won't store it to separate file, but merge to the graph
#eschenburg_lehrgedicht_beschreibende_poesie_epistel.store(folder = "out", filename = "eschenburg_skos_collections")
eschenburg_g = eschenburg_g + eschenburg_lehrgedicht_beschreibende_poesie_epistel.dump()

In [16]:
#print(eschenburg_g.serialize())
eschenburg_g.serialize(destination="out/eschenburg.ttl")

<Graph identifier=N16b71fe2773a4902bc76122051820ce5 (<class 'rdflib.graph.Graph'>)>

This should go to Graph:
`https://genre.clscor.io/eschenburg/` Currently I am manually uploading it

## Friedrich Bouterwek: Aesthetik. Zweiter Theil. 2. Auflage. Göttingen 1815

* Lyrische Dichtungsarten
    1. Das Lied
    2. Die Ode
    3. Die romantische Canzone, das lyrische Sonett usw.
    4. Die Elegie
    5. Die lyrische Epistel
* Didaktische Dichtungsarten
    1. Die didaktische Satyre
    2. Die didaktische Epistel
    3. Das Spruchgedicht
    4. Das ausführliche und vorzugsweise so genannte Lehrgedicht
* Epische Dichtungsarten
    1. Mancherlei erzählende Gedichte
    2. Die epische Romanze und Ballade
    3. Die Epopöe
* Dramatische Dichtungsarten
    1. Mancherlei bisher verkannte, oder noch zu wenig cultivirte dramatische Dichtungsarten
    2. Das Melodram und die Oper
    3. Das Lustspiel
    4. Das Trauerspiel 
* Fünfte oder Ergänzungs-Classe
    1. Das Hirtengedicht und die idyllische Poesie
    2. Das beschreibende Gedicht
    3. Das Epigramm
    4. Die äsopische Fabel
    5. Der Roman

In [17]:
# create the graph to collect the pieces of the SKOS
bouterwek_g = Graph()

In [18]:
bouterwek_raw_data = [

    {
        "id" : "lyrische_dichtungsart",
        "label" : "Lyrische Dichtungsart",
        "narrower" : [

            {
                "id" : "lied",
                "label" : "Lied"
            },
            {
                "id" : "ode",
                "label" : "Ode"
            },
            {
              "id" : "romantische_canzone",
                "label" : "Romantische Canzone"
            },
            {
                "id" : "lyrisches_sonett",
                "label" : "Lyrisches Sonett"
            },
            {
                "id" : "elegie",
                "label" : "Elegie"
            },
            {
                "id" : "lyrische_epistel",
                "label" : "Lyrische Epistel"
            }
        ]
    },
    {
        "id" : "didaktische_dichtungsart",
        "label" : "Didaktische Dichtungsart",
        "narrower" : [
            {
                "id" : "didaktische_satyre",
                "label" : "Didaktische Satyre"
            },
            {
                "id" : "didaktische_epistel",
                "label" : "Didaktische Epistel"
            },
            {
                "id" : "spruchgedicht",
                "label" : "Spruchgedicht"
            },
            {
                "id" : "lehrgedicht",
                "label" : "Lehrgedicht"
            }
            
        ]
    },
    {
        "id" : "epische_dichtungsart",
        "label" : "Epische Dichtungsart",
        "narrower" : [
            {
                "id": "erzaehlendes_gedicht",
            "label" : "Erzählendes Gedicht"
            },
            {
                "id" : "epische_romanze",
                "label" : "Epische Romanze" 
            },
            {
                "id" : "ballade",
                "label" : "Ballade"
            },
            {
                "id": "epopoee",
                "label" : "Epopöe"
            }
            
            
        ]
    },
    {
        "id" : "dramatische_dichtungsart",
        "label" : "Dramatische Dichtungsart",
        "narrower" : [
            
            {
                "id" : "melodram",
                "label" : "Melodram"
            },
            {
                "id" : "oper",
                "label" : "Oper"
            },
            {
                "id" : "lustspiel",
                "label" : "Lustspiel"
            },
            {
                "id" : "trauerspiel",
                "label" : "Trauerspiel"
            }
            
        ]
    },
    {
        "id" : "hirtengedicht",
        "label" : "Das Hirtengedicht und die idyllische Poesie"
    },
    {
        "id" : "beschreibendes_gedicht",
        "label" : "Beschreibendes Gedicht"
    },
    {
        "id" : "epigramm",
        "label" : "Epigramm"
    },
    {
        "id" : "aesopische_fabel",
        "label" : "Äsopische Fabel"
    },
    {
        "id" : "roman",
        "label" : "Roman"
    }
]

In [19]:
bouterwek_base_uri = "https://genre.clscor.io/bouterwek/"

In [20]:
bouterwek = SkosConceptScheme(uri=bouterwek_base_uri)

In [21]:
bouterwek.rdfs_label("Bouterwek: Aesthetik. Zweiter Theil. 2. Auflage. Göttingen 1815 [Gattungssystem]","de")

True

In [22]:
# create this for bouterwek

for item in bouterwek_raw_data:
    top_concept = create_concept_from_dict(item, base_uri=bouterwek_base_uri,concept_scheme=bouterwek)
    bouterwek.skos_has_top_concept(top_concept)

In [23]:
# store it
#bouterwek.store(folder = "out", filename = "bouterwek_concepts")
bouterwek_g = bouterwek_g + bouterwek.dump()

### Bouterwek Collections
e.g. the infamous "Fünfte Classe"

In [24]:
bouterweg_fuenfte_classe_raw = {
        "id" : "fuenfte_classe",
        "label": "Fünfte oder Ergänzungs-Classe",
        "narrower" : [
            {
                "id" : "hirtengedicht",
                "label" : "Das Hirtengedicht und die idyllische Poesie"
            },
            {
                "id" : "beschreibendes_gedicht",
                "label" : "Beschreibendes Gedicht"
            },
            {
                "id" : "epigramm",
                "label" : "Epigramm"
            },
            {
                "id" : "aesopische_fabel",
                "label" : "Äsopische Fabel"
            },
            {
                "id" : "roman",
                "label" : "Roman"
            }
                
        ]
    }

In [25]:
bouterwek_fuenfte_classe = SkosCollection(uri=bouterwek_base_uri+"collection/ergaenzungsclasse")
bouterwek_fuenfte_classe.skos_pref_label("Fünfte oder Ergänzungs-Classe","de")

for item in bouterweg_fuenfte_classe_raw["narrower"]:
    bouterwek_fuenfte_classe.skos_member(uris=[bouterwek_base_uri + item["id"]])

In [26]:
print(bouterwek_fuenfte_classe.serialize())

@prefix skos: <http://www.w3.org/2004/02/skos/core#> .

<https://genre.clscor.io/bouterwek/collection/ergaenzungsclasse> a skos:Collection ;
    skos:member <https://genre.clscor.io/bouterwek/aesopische_fabel>,
        <https://genre.clscor.io/bouterwek/beschreibendes_gedicht>,
        <https://genre.clscor.io/bouterwek/epigramm>,
        <https://genre.clscor.io/bouterwek/hirtengedicht>,
        <https://genre.clscor.io/bouterwek/roman> ;
    skos:prefLabel "Fünfte oder Ergänzungs-Classe"@de .




In [27]:
# add to main graph
bouterwek_g = bouterwek_g + bouterwek_fuenfte_classe.dump() 

In [28]:
#Die romantische Canzone, das lyrische Sonett usw.

In [29]:
bouterwek_romantische_canzone_col = SkosCollection(uri=bouterwek_base_uri+ "collection/canzone_sonett_usw")
bouterwek_romantische_canzone_col.skos_pref_label("Die romantische Canzone, das lyrische Sonett usw.","de")
bouterwek_romantische_canzone_member_uris = [bouterwek_base_uri + "lyrisches_sonett", 
                                             bouterwek_base_uri + "romantische_canzone"]
bouterwek_romantische_canzone_col.skos_member(uris=bouterwek_romantische_canzone_member_uris)
print(bouterwek_romantische_canzone_col.serialize())

@prefix skos: <http://www.w3.org/2004/02/skos/core#> .

<https://genre.clscor.io/bouterwek/collection/canzone_sonett_usw> a skos:Collection ;
    skos:member <https://genre.clscor.io/bouterwek/lyrisches_sonett>,
        <https://genre.clscor.io/bouterwek/romantische_canzone> ;
    skos:prefLabel "Die romantische Canzone, das lyrische Sonett usw."@de .




In [30]:
bouterwek_g = bouterwek_g + bouterwek_romantische_canzone_col.dump()

In [31]:
# Mancherlei erzählende Gedichte
mancherlei_erzaehlende_gedichte = SkosCollection(uri=bouterwek_base_uri+"collection/mancherlei_erzaehlende_gedichte")
mancherlei_erzaehlende_gedichte.skos_pref_label("Mancherlei erzählende Gedichte", "de")
bouterwek_g = bouterwek_g + mancherlei_erzaehlende_gedichte.dump()

In [32]:
# Mancherlei bisher verkannte, oder noch zu wenig cultivirte dramatische Dichtungsarten
mancherlei_dramatische_dichtungsarten = SkosCollection(uri=bouterwek_base_uri+"collection/mancherlei_dramatische_dichtungsarten")
mancherlei_dramatische_dichtungsarten.skos_pref_label("Mancherlei bisher verkannte, oder noch zu wenig cultivirte dramatische Dichtungsarten", "de")
bouterwek_g = bouterwek_g + mancherlei_dramatische_dichtungsarten.dump()

In [33]:
# Das Melodram und die Oper
melodram_und_oper = SkosCollection(uri=bouterwek_base_uri+"collection/melodram_oper")
melodram_und_oper.skos_pref_label("Das Melodram und die Oper","de")
melodram_und_oper_uris = [bouterwek_base_uri+"melodram",bouterwek_base_uri+"oper"]
melodram_und_oper.skos_member(uris=melodram_und_oper_uris)
bouterwek_g = bouterwek_g + melodram_und_oper.dump()

### Some manual interventions to Bouterwek

In [34]:
# 4. Das ausführliche und vorzugsweise so genannte Lehrgedicht
bouterwek_lehrgedicht = SkosConcept(uri=bouterwek_base_uri+"lehrgedicht")
bouterwek_lehrgedicht.skos_alt_label("Das ausführliche und vorzugsweise so genannte Lehrgedicht","de")
bouterwek_g = bouterwek_g + bouterwek_lehrgedicht.dump()

In [35]:
bouterwek_g.serialize(destination="out/bouterwek.ttl")

<Graph identifier=Nd4382ed95ad9484d92f44338994e2410 (<class 'rdflib.graph.Graph'>)>

Upload to `https://genre.clscor.io/bouterwek/`

## Goehte: West-Östlicher Divan

Johann Wolfgang von Goethe: West-Östlicher Divan. Stuttgart 1819.

It is not entirely clear how we are going to represent this genre system as SKOS; "Naturform" and "Dichtsart" as `skos:Concept` and `skos:topConcept` of the `skos:ConceptScheme`. An alternative would be to have the "Dichtarten" as a `skos:Collection`.

Peer says:
> Natrurformen: Concept, Dichtarten: Collection

		
* Naturformen	
    * Epik
    * Lyrik
    * Drama
* Dichtarten	
    * Allegorie
    * Ballade
    * Cantate
    * Drama
    * Elegie
    * Epigramm
    * Epistel
    * Epopöe
    * Erzählung
	* Fabel
	* Heroide
	* Idylle
	* Lehrgedicht
	* Ode
	* Parodie
	* Roman
	* Romanze
	* Satire

In [36]:
# create a graph to collect generated pieces of the SKOS
goethe_g = Graph()

In [37]:
goethe_base_uri = "https://genre.clscor.io/goethe/"

In [38]:
goethe_scheme = SkosConceptScheme(uri=goethe_base_uri)
goethe_scheme.rdfs_label("Goethe: West-Östlicher Divan [Gattungssystem]", "de")

True

In [39]:
goethe_g = goethe_g + goethe_scheme.dump()

In [40]:
goethe_dichtarten_raw = [
    "Allegorie", 
    "Ballade",
    "Cantate",
    "Drama",
    "Elegie",
    "Epigramm",
    "Epistel",
    "Epopöe",
    "Erzählung",
    "Fabel",
    "Heroide",
    "Idylle",
    "Lehrgedicht",
    "Ode",
    "Parodie",
    "Roman",
    "Romanze",
    "Satire"
]

# There is "Drama" as well in "Naturformen" is Drama (Naturform) something else than Drama (Dichtart)?

In [41]:
# Create Dichtarten as a skos:Collection
goethe_dichtarten_collection = SkosCollection(uri=goethe_base_uri+"collection/dichtarten")
goethe_dichtarten_collection.skos_pref_label("Dichtarten","de")

True

In [42]:
for item in goethe_dichtarten_raw:
    concept_uri = goethe_base_uri + item.lower().replace("ö","oe").replace("ä","ae")
    concept = SkosConcept(uri=concept_uri)
    concept.skos_pref_label(item,"de")
    concept.skos_in_scheme(goethe_scheme)
    # add it to the dichtarten collection
    goethe_dichtarten_collection.skos_member(concept)

In [43]:
# add this to the graph collection the parts of the skos
goethe_g = goethe_g + goethe_dichtarten_collection.dump()

In [44]:
# Naturformen as Collection
goethe_naturformen_collection = SkosCollection(uri=goethe_base_uri+"collection/naturformen")
goethe_naturformen_collection.skos_pref_label("Naturformen","de")

True

In [45]:
#Epik, Lyrik, Drama
# because of "Drama" will add a prefix "naturform_" to the id in the uri
goethe_naturformen_raw = ["Epik", "Lyrik", "Drama"]

for item in goethe_naturformen_raw:
    concept_uri = goethe_base_uri + "naturform_" + item.lower()
    concept = SkosConcept(uri=concept_uri)
    concept.skos_pref_label(item,"de")
    concept.skos_in_scheme(goethe_scheme)
    goethe_naturformen_collection.skos_member(concept)

In [46]:
goethe_g = goethe_g + goethe_naturformen_collection.dump()

In [47]:
print(goethe_g.serialize())

@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix skos: <http://www.w3.org/2004/02/skos/core#> .

<https://genre.clscor.io/goethe/collection/dichtarten> a skos:Collection ;
    skos:member <https://genre.clscor.io/goethe/allegorie>,
        <https://genre.clscor.io/goethe/ballade>,
        <https://genre.clscor.io/goethe/cantate>,
        <https://genre.clscor.io/goethe/drama>,
        <https://genre.clscor.io/goethe/elegie>,
        <https://genre.clscor.io/goethe/epigramm>,
        <https://genre.clscor.io/goethe/epistel>,
        <https://genre.clscor.io/goethe/epopoee>,
        <https://genre.clscor.io/goethe/erzaehlung>,
        <https://genre.clscor.io/goethe/fabel>,
        <https://genre.clscor.io/goethe/heroide>,
        <https://genre.clscor.io/goethe/idylle>,
        <https://genre.clscor.io/goethe/lehrgedicht>,
        <https://genre.clscor.io/goethe/ode>,
        <https://genre.clscor.io/goethe/parodie>,
        <https://genre.clscor.io/goethe/roman>,
       

In [48]:
goethe_g.serialize(destination="out/goethe.ttl")

<Graph identifier=N3b2a1d046a8a4bcba472ae9dc3f149de (<class 'rdflib.graph.Graph'>)>

Upload to named graph `https://genre.clscor.io/goethe/`